In [82]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np

In [84]:
## 1. Python 라이브러리 함수를 사용하여, 엑셀 파일을 불러오고, DataFrame을 출력해주세요.

# 사용한 데이터 파일들 데이터프레임 형태로 불러오기
crime_df = pd.read_excel("관서별_5대범죄_발생_및_검거.xlsx")  # read_excel 사용, sheet하나 밖에 없는 excel파일이라 sheet 옵션은 사용 안씀 (defaultr값)
pop_df = pd.read_csv("pop_kor.csv")   # read_csv 사용

In [86]:
## 2. 각 경찰서('관서명')를 해당 구 이름으로 매핑하여 '구별'이라는 새로운 column을 생성하고, DataFrame을 출력해주세요

gu_data = {'서대문서': '서대문구', '수서서': '강남구', '강서서': '강서구', '서초서': '서초구',
'서부서': '은평구', '중부서': '중구', '종로서': '종로구', '남대문서': '중구',
'혜화서': '종로구', '용산서': '용산구', '성북서': '성북구', '동대문서': '동대문구',
'마포서': '마포구', '영등포서': '영등포구', '성동서': '성동구', '동작서': '동작구',
'광진서': '광진구', '강북서': '강북구', '금천서': '금천구', '중랑서': '중랑구',
'강남서': '강남구', '관악서': '관악구', '강동서': '강동구', '종암서': '성북구', 
'구로서': '구로구', '양천서': '양천구', '송파서': '송파구', '노원서': '노원구', 
'방배서': '서초구', '은평서': '은평구', '도봉서': '도봉구'}   # {관서명 : 구이름} 형태의 딕셔너리 데이터 생성

crime_df['구별'] = crime_df['관서명'].map(gu_data)  # 구 데이터는 딕셔너리 형태이므로 map()를 이용해 매핑 진행
crime_df = crime_df.fillna('구 없음')  # 매핑 진행 후, crime_df.loc["계"] 행의 crime_df['구별']에 NaN 하나만 있으므로, 복잡하게 생각하지 않고 fillna('구 없음')을 이용해 NaN을 '구 없음'으로 채워줌 

In [89]:
## 3. pivot_table 을 사용하여 관서별 데이터를 구별 데이터로 변경하고, 같은 구의 경우에는 sum을 적용하여 더해주세요, (index: 관서명 -> 구별) 

pivot_df = pd.pivot_table(crime_df, index = "구별", aggfunc = 'sum')  # pivot이 아니라 pivot_table을 통해 구별로 groupby하고 같은 구로 묶인 데이터는 aggfunc 옵션을 sum으로 해줌으로써 더함.
pivot_df = pivot_df.drop(columns = '관서명')   # 예시 데이터에는 pivot_df['관서명'] 데이터가 없으므로, drop을 이용해서 해당 열 제거

In [91]:
## 4. 구 없음 행은 drop을 활용하여 삭제해주세요.

pivot_df = pivot_df.drop(['구 없음'], axis = 0)  # axis = 0 으로 행을 제거, index에서 '구 없음' 지정

In [93]:
## 5. 각 범죄 별로 검거율을 계산하고, 각 검거율 데이터를 column을 DataFrame에 추가해주세요.

for crime in ['강간', '강도', '살인', '절도', '폭력']:   # 반복된 형태를 작성하고 달라지는 부분만 넣는 형틱으로 반복작업을 진행 (for문)
    pivot_df[f'{crime}검거율'] = pivot_df[f'{crime}(검거)'] / pivot_df[f'{crime}(발생)'] * 100    # 검거율을 계산하여, 검거율 열을 만들어서 데이터를 넣어줌. Series 계산이므로 각 행에 맞춰서 계산됨

pivot_df['검거율'] = pivot_df['소계(검거)'] / pivot_df['소계(발생)'] * 100  # 전체 검거율은 형식이 달라서 따로 진행하였음.

In [95]:
## 6. 필요없는 column을 del을 사용하여 삭제해주세요.

for i in ['강간', '강도', '살인', '절도', '폭력', '소계']:   # for문과 if 문을 이용해 반복 작업 진행.
    if i == '소계':    # if문을 이용해 소계 데이터 열만 검거와 발생 모두 제거하고, 나머지는 검거 데이터 열만 삭제
        del pivot_df[f'{i}(검거)']
        del pivot_df[f'{i}(발생)']
    else:
        del pivot_df[f'{i}(검거)']

In [97]:
# 7. DataFrame 컬럼명을 rename을 사용하여 변경해주세요.

rename_data = {'강간(발생)':'강간',   # 변환할 이름에 대한 정보를 딕셔너리 형태로 정의
'강도(발생)':'강도',
'살인(발생)':'살인',
'절도(발생)':'절도',
'폭력(발생)':'폭력'}

pivot_df = pivot_df.rename(columns = rename_data)   # rename을 이용해, columns 옵션을 통해 열 이름을 변환하는 것을 알려주고, 변환할 정도는 정의한 딕셔너리를 넣어줌.

In [98]:
print(pivot_df)

       강간  강도  살인    절도    폭력       강간검거율       강도검거율       살인검거율      절도검거율  \
구별                                                                             
강남구   449  21  13  3850  4284   77.728285   85.714286   76.923077  42.857143   
강동구   156   6   4  2366  2712   78.846154  133.333333   75.000000  33.347422   
강북구   153  14   7  1434  2649   82.352941   92.857143  114.285714  43.096234   
강서구   262  13   7  2096  3207   72.900763  100.000000  114.285714  60.114504   
관악구   320  12   9  2706  3298   69.062500  116.666667   88.888889  30.561715   
광진구   240  14   4  3026  2625   91.666667  185.714286  100.000000  42.200925   
구로구   281  15   8  2335  3007   58.362989   73.333333   75.000000  38.072805   
금천구   151   6   3  1567  2054   80.794702  100.000000  133.333333  56.668794   
노원구   197   7  10  2193  2723   61.421320  100.000000  100.000000  36.525308   
도봉구   102   9   3  1063  1487  103.921569  111.111111  100.000000  44.967074   
동대문구  173  13   5  1981  2548   84.39306

In [101]:
## 도전 1. Python 라이브러리 함수를 사용하여 인구 데이터(pop_kor.csv) 파일을 불러오고, DataFrame을 출력해주세요.
##    - Quiz에서 수행한 DataFrame의 구별 index를 기준으로 merge를 할 것이므로, index를 셋팅해서 불러와주세요.

pop_df = pd.read_csv("pop_kor.csv", index_col = '구별')

In [103]:
# 도전 2. join을 사용하여 Quiz에서 수행한 DataFrame과 인구 데이터 DataFrame을 merge하고, DataFrame을 출력해주세요.

merged_df = pivot_df.join(pop_df)
print(merged_df)

       강간  강도  살인    절도    폭력       강간검거율       강도검거율       살인검거율      절도검거율  \
구별                                                                             
강남구   449  21  13  3850  4284   77.728285   85.714286   76.923077  42.857143   
강동구   156   6   4  2366  2712   78.846154  133.333333   75.000000  33.347422   
강북구   153  14   7  1434  2649   82.352941   92.857143  114.285714  43.096234   
강서구   262  13   7  2096  3207   72.900763  100.000000  114.285714  60.114504   
관악구   320  12   9  2706  3298   69.062500  116.666667   88.888889  30.561715   
광진구   240  14   4  3026  2625   91.666667  185.714286  100.000000  42.200925   
구로구   281  15   8  2335  3007   58.362989   73.333333   75.000000  38.072805   
금천구   151   6   3  1567  2054   80.794702  100.000000  133.333333  56.668794   
노원구   197   7  10  2193  2723   61.421320  100.000000  100.000000  36.525308   
도봉구   102   9   3  1063  1487  103.921569  111.111111  100.000000  44.967074   
동대문구  173  13   5  1981  2548   84.39306

In [105]:
# 3. 새롭게 merge 된 DataFrame에서 검거율 기준으로 오름차순 정렬 후, DataFrame을 출력해주세요.

final_df = merged_df.sort_values(by = '검거율', ascending = True)
print(final_df)

       강간  강도  살인    절도    폭력       강간검거율       강도검거율       살인검거율      절도검거율  \
구별                                                                             
관악구   320  12   9  2706  3298   69.062500  116.666667   88.888889  30.561715   
동작구   285   9   5  1865  1910   48.771930   55.555556  100.000000  35.442359   
중구    170   9   3  2548  2224   65.294118   66.666667   66.666667  33.712716   
강동구   156   6   4  2366  2712   78.846154  133.333333   75.000000  33.347422   
영등포구  295  22  14  2964  3572   62.033898   90.909091   85.714286  32.995951   
송파구   220  13  11  3239  3295   80.909091   76.923077   90.909091  34.856437   
마포구   294  14   8  2555  2983   84.013605   71.428571  100.000000  31.819961   
구로구   281  15   8  2335  3007   58.362989   73.333333   75.000000  38.072805   
양천구   120   6   3  1890  2509   87.500000   50.000000  166.666667  35.555556   
종로구   211  11   6  2184  2293   76.303318   81.818182   83.333333  38.324176   
광진구   240  14   4  3026  2625   91.66666